In [1]:
from transformers import BertTokenizer, BertForSequenceClassification

def personality_detection(text):
    tokenizer = BertTokenizer.from_pretrained("Minej/bert-base-personality")
    model = BertForSequenceClassification.from_pretrained("Minej/bert-base-personality")

    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    outputs = model(**inputs)
    predictions = outputs.logits.squeeze().detach().numpy()

    label_names = ['Extroversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness']
    result = {label_names[i]: predictions[i] for i in range(len(label_names))}

    return result

In [2]:
import PyPDF2
import nltk
from nltk.tokenize import word_tokenize

def pdf_to_tokens(pdf_path):
    # Open the PDF file
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)

        all_text = ''
        # Extract text from each page
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            all_text += page.extract_text() + ' '  # Append space after each page's text

        # Tokenize the text
        tokens = word_tokenize(all_text)

        # Split the tokens into chunks of 1000 tokens each
        chunks = [' '.join(tokens[i:i + 1000]) for i in range(0, len(tokens), 1000)]

        return chunks

# Example usage
pdf_path = 'test_pdf.pdf'
chunks = pdf_to_tokens(pdf_path)
print(chunks)

['PERSPECTIVE OPEN ACCESS Can Generative AI improve social science ? Christopher A. Baila , b , c,1 Edited by David Lazer , Northeastern University , Boston , MA ; received September 7 , 2023 ; accepted April 5 , 2024 , by Editorial Board Member Mark Granovetter Generative AI that can produce realistic text , images , and other human-like outputs is currently transforming many diﬀerent industries . Yet it is not yet known how such tools might influence social science research . I argue Generative AI has the potential to improve survey re- search , online experiments , automated content analyses , agent-based models , and other techniques commonly used to study human behavior . In the second section of this article , I discuss the many limitations of Generative . I examine how bias in the data used to train these tools can negatively impact social science research—as well as a range of other challenges related to ethics , replication , environmental impact , and the proliferation of low

In [4]:
results = []
# loop through the chunks and get the personality detection
for chunk in chunks:
    results.append(personality_detection(chunk)) 

In [5]:
# Initialize a dictionary to store the sum of traits
sums = {trait: 0 for trait in results[0]}  # Use the first item to get all trait names

# Sum values for each trait
for person in results:
    for trait in person:
        sums[trait] += person[trait]

# Calculate the average for each trait
averages = {trait: sum_val / len(results) for trait, sum_val in sums.items()}

print("Average values for each trait:")
for trait, average in averages.items():
    print(f"{trait}: {average}")

Average values for each trait:
Extroversion: 0.013186319598129817
Neuroticism: 0.11834167476211276
Agreeableness: 0.07243931612798146
Conscientiousness: -0.07180587095873696
Openness: 0.34166261340890614
